In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gc

import category_encoders as ce

root='C:/Users/luthf/Documents/DS/BasketAnalysis/'

In [2]:
orders = pd.read_csv(root + 'orders.csv', 
                 dtype={
                        'order_id': np.int32,
                        'user_id': np.int64,
                        'eval_set': 'category',
                        'order_number': np.int16,
                        'order_dow': np.int8,
                        'order_hour_of_day': np.int8,
                        'days_since_prior_order': np.float32})


order_products_train = pd.read_csv(root + 'order_products__train.csv', 
                                 dtype={
                                        'order_id': np.int32,
                                        'product_id': np.uint16,
                                        'add_to_cart_order': np.int16,
                                        'reordered': np.int8})

order_products_prior = pd.read_csv(root + 'order_products__prior.csv', 
                                 dtype={
                                        'order_id': np.int32,
                                        'product_id': np.uint16,
                                        'add_to_cart_order': np.int16,
                                        'reordered': np.int8})

product_level = pd.read_pickle(root + 'prod_level.pkl')

user_level = pd.read_pickle(root + 'user_level.pkl')

user_product_features = pd.read_pickle(root + 'user_product_features.pkl')

products = pd.read_csv(root +'products.csv')

aisles = pd.read_csv(root + 'aisles.csv')

departments = pd.read_csv(root + 'departments.csv')

In [3]:
user_level.head()

,user_id,user_avg_dow_order,user_std_dow_order,user_avg_hour_order,user_std_hour_order,user_avg_prior_order,user_std_prior_order,user_total_order,user_total_product_order,user__unique_product_order,user_total_reorder,user_overal_reorder,avg_order_size,mean_reorder_size,orders_3,orders_2,orders_1,reorder_3,reorder_2,reorder_1
0,1,2.644068,1.256194,10.542373,3.500355,18.542374,10.559066,10,59,18,41.0,0.694915,5.900000,0.705833,6,6,9,0.666667,1.0,0.666667
1,2,2.005128,0.971222,10.441026,1.649854,14.902564,9.671712,14,195,102,93.0,0.476923,13.928571,0.447961,19,9,16,0.578947,0.0,0.625000
2,3,1.011364,1.245630,16.352273,1.454599,10.181818,5.867395,12,88,33,55.0,0.625000,7.333333,0.658817,6,5,6,0.833333,1.0,1.000000
3,4,4.722222,0.826442,13.111111,1.745208,11.944445,9.973330,5,18,17,1.0,0.055556,3.600000,0.028571,7,2,3,0.142857,0.0,0.000000
4,5,1.621622,1.276961,15.729730,2.588958,10.189189,7.600577,4,37,23,14.0,0.378378,9.250000,0.377778,9,5,12,0.444444,0.4,0.666667


In [4]:
train_order=orders.merge(order_products_train,on='order_id',how='inner')

In [5]:
train_order.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,1187899,1,train,11,4,8,14.0,196,1,1
1,1187899,1,train,11,4,8,14.0,25133,2,1
2,1187899,1,train,11,4,8,14.0,38928,3,1
3,1187899,1,train,11,4,8,14.0,26405,4,1
4,1187899,1,train,11,4,8,14.0,39657,5,1


In [6]:
train_order.drop(columns=['add_to_cart_order','eval_set','order_id'],axis=1,inplace=True)

In [7]:
train_user=train_order.user_id.unique()

In [8]:
df=user_product_features[user_product_features.user_id.isin(train_user)]

In [9]:
df.head()

,user_id,product_id,avg_add_to_cart_by_user,avg_day_since_prior_order,user_total_order_product,user_total_reorder_product,user_percentage_reorder_product,last_order_in,is_reorder_3,is_reorder_2,is_reorder_1
0,1,196,1.400000,17.600000,10,9,0.900000,10,1.0,1.0,1.0
1,1,10258,3.333333,19.555555,9,8,0.888889,10,1.0,1.0,1.0
2,1,10326,5.000000,28.000000,1,0,0.000000,5,0.0,0.0,0.0
3,1,12427,3.300000,17.600000,10,9,0.900000,10,1.0,1.0,1.0
4,1,13032,6.333333,21.666666,3,2,0.666667,10,1.0,0.0,0.0


In [10]:
df=df.merge(train_order, on = ['user_id', 'product_id'], how = 'outer')

In [11]:
df.head()

,user_id,product_id,avg_add_to_cart_by_user,avg_day_since_prior_order,user_total_order_product,user_total_reorder_product,user_percentage_reorder_product,last_order_in,is_reorder_3,is_reorder_2,is_reorder_1,order_number,order_dow,order_hour_of_day,days_since_prior_order,reordered
0,1,196,1.400000,17.600000,10.0,9.0,0.900000,10.0,1.0,1.0,1.0,11.0,4.0,8.0,14.0,1.0
1,1,10258,3.333333,19.555555,9.0,8.0,0.888889,10.0,1.0,1.0,1.0,11.0,4.0,8.0,14.0,1.0
2,1,10326,5.000000,28.000000,1.0,0.0,0.000000,5.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
3,1,12427,3.300000,17.600000,10.0,9.0,0.900000,10.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN
4,1,13032,6.333333,21.666666,3.0,2.0,0.666667,10.0,1.0,0.0,0.0,11.0,4.0,8.0,14.0,1.0


In [12]:
df.isnull().sum()

user_id                                  0
product_id                               0
avg_add_to_cart_by_user             555793
avg_day_since_prior_order           555793
user_total_order_product            555793
user_total_reorder_product          555793
user_percentage_reorder_product     555793
last_order_in                       555793
is_reorder_3                        555793
is_reorder_2                        555793
is_reorder_1                        555793
order_number                       7645837
order_dow                          7645837
order_hour_of_day                  7645837
days_since_prior_order             7645837
reordered                          7645837
dtype: int64

In [13]:
df.order_number.fillna(df.groupby('user_id')['order_number'].transform('mean'), inplace = True)
df.order_dow.fillna(df.groupby('user_id')['order_dow'].transform('mean'), inplace = True)
df.order_hour_of_day.fillna(df.groupby('user_id')['order_hour_of_day'].transform('mean'), inplace = True)
df.days_since_prior_order.fillna(df.groupby('user_id')['days_since_prior_order'].\
                                                             transform('mean'), inplace = True)

In [14]:
df.isnull().sum()

user_id                                  0
product_id                               0
avg_add_to_cart_by_user             555793
avg_day_since_prior_order           555793
user_total_order_product            555793
user_total_reorder_product          555793
user_percentage_reorder_product     555793
last_order_in                       555793
is_reorder_3                        555793
is_reorder_2                        555793
is_reorder_1                        555793
order_number                             0
order_dow                                0
order_hour_of_day                        0
days_since_prior_order                   0
reordered                          7645837
dtype: int64

In [15]:
df.reordered.value_counts()

1.0    828824
0.0    555793
Name: reordered, dtype: int64

In [16]:
df = df[df.reordered != 0]

In [17]:
df.reordered.fillna(0, inplace = True)

df.isnull().sum()

user_id                            0
product_id                         0
avg_add_to_cart_by_user            0
avg_day_since_prior_order          0
user_total_order_product           0
user_total_reorder_product         0
user_percentage_reorder_product    0
last_order_in                      0
is_reorder_3                       0
is_reorder_2                       0
is_reorder_1                       0
order_number                       0
order_dow                          0
order_hour_of_day                  0
days_since_prior_order             0
reordered                          0
dtype: int64

### merging user and product features

In [18]:
df=df.merge(product_level,on='product_id',how='left')
df=df.merge(user_level,on='user_id',how='left')

In [19]:
df.head()

,user_id,product_id,avg_add_to_cart_by_user,avg_day_since_prior_order,user_total_order_product,user_total_reorder_product,user_percentage_reorder_product,last_order_in,is_reorder_3,is_reorder_2,...,user_total_reorder,user_overal_reorder,avg_order_size,mean_reorder_size,orders_3,orders_2,orders_1,reorder_3,reorder_2,reorder_1
0,1,196,1.400000,17.600000,10.0,9.0,0.900000,10.0,1.0,1.0,...,41.0,0.694915,5.9,0.705833,6,6,9,0.666667,1.0,0.666667
1,1,10258,3.333333,19.555555,9.0,8.0,0.888889,10.0,1.0,1.0,...,41.0,0.694915,5.9,0.705833,6,6,9,0.666667,1.0,0.666667
2,1,10326,5.000000,28.000000,1.0,0.0,0.000000,5.0,0.0,0.0,...,41.0,0.694915,5.9,0.705833,6,6,9,0.666667,1.0,0.666667
3,1,12427,3.300000,17.600000,10.0,9.0,0.900000,10.0,1.0,1.0,...,41.0,0.694915,5.9,0.705833,6,6,9,0.666667,1.0,0.666667
4,1,13032,6.333333,21.666666,3.0,2.0,0.666667,10.0,1.0,0.0,...,41.0,0.694915,5.9,0.705833,6,6,9,0.666667,1.0,0.666667


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8474661 entries, 0 to 8474660
Data columns (total 65 columns):
 #   Column                           Dtype  
---  ------                           -----  
 0   user_id                          int64  
 1   product_id                       uint64 
 2   avg_add_to_cart_by_user          float64
 3   avg_day_since_prior_order        float32
 4   user_total_order_product         float64
 5   user_total_reorder_product       float64
 6   user_percentage_reorder_product  float64
 7   last_order_in                    float64
 8   is_reorder_3                     float64
 9   is_reorder_2                     float64
 10  is_reorder_1                     float64
 11  order_number                     float64
 12  order_dow                        float64
 13  order_hour_of_day                float64
 14  days_since_prior_order           float32
 15  reordered                        float64
 16  mean_add_to_cart                 float64
 17  total_or

In [21]:
product_level.head()

,product_id,mean_add_to_cart,total_order,total_reorder,reordered_percentage,unique_user,first_buy,second_buy,aisle_mean_add_to_cart,aisle_std_add_to_cart,...,aisle_7,department_mean_add_to_cart,department_std_add_to_cart,department_reordered_percentage,department_unique_user,department_0,department_1,department_2,department_3,department_4
0,1,5.801836,1852,1136.0,0.613391,716,716,276,9.253092,7.845107,...,1,9.187743,7.692492,0.574180,174219,0,0,0,0,1
1,2,9.888889,90,12.0,0.133333,78,78,8,9.996181,7.994790,...,0,9.593425,7.875241,0.346721,172755,0,0,0,1,0
2,3,6.415162,277,203.0,0.732852,74,74,36,8.519846,7.818249,...,1,6.976699,6.711172,0.653460,172795,0,0,0,1,1
3,4,9.507599,329,147.0,0.446809,182,182,64,9.207741,7.435740,...,0,8.996414,7.393502,0.541885,163233,0,0,1,0,0
4,5,6.466667,15,9.0,0.600000,6,6,4,10.297600,8.187047,...,1,9.593425,7.875241,0.346721,172755,0,0,0,1,0


In [22]:
user_level.head()

,user_id,user_avg_dow_order,user_std_dow_order,user_avg_hour_order,user_std_hour_order,user_avg_prior_order,user_std_prior_order,user_total_order,user_total_product_order,user__unique_product_order,user_total_reorder,user_overal_reorder,avg_order_size,mean_reorder_size,orders_3,orders_2,orders_1,reorder_3,reorder_2,reorder_1
0,1,2.644068,1.256194,10.542373,3.500355,18.542374,10.559066,10,59,18,41.0,0.694915,5.900000,0.705833,6,6,9,0.666667,1.0,0.666667
1,2,2.005128,0.971222,10.441026,1.649854,14.902564,9.671712,14,195,102,93.0,0.476923,13.928571,0.447961,19,9,16,0.578947,0.0,0.625000
2,3,1.011364,1.245630,16.352273,1.454599,10.181818,5.867395,12,88,33,55.0,0.625000,7.333333,0.658817,6,5,6,0.833333,1.0,1.000000
3,4,4.722222,0.826442,13.111111,1.745208,11.944445,9.973330,5,18,17,1.0,0.055556,3.600000,0.028571,7,2,3,0.142857,0.0,0.000000
4,5,1.621622,1.276961,15.729730,2.588958,10.189189,7.600577,4,37,23,14.0,0.378378,9.250000,0.377778,9,5,12,0.444444,0.4,0.666667


In [23]:
df.shape

(8474661, 65)

In [24]:
df.head()

,user_id,product_id,avg_add_to_cart_by_user,avg_day_since_prior_order,user_total_order_product,user_total_reorder_product,user_percentage_reorder_product,last_order_in,is_reorder_3,is_reorder_2,...,user_total_reorder,user_overal_reorder,avg_order_size,mean_reorder_size,orders_3,orders_2,orders_1,reorder_3,reorder_2,reorder_1
0,1,196,1.400000,17.600000,10.0,9.0,0.900000,10.0,1.0,1.0,...,41.0,0.694915,5.9,0.705833,6,6,9,0.666667,1.0,0.666667
1,1,10258,3.333333,19.555555,9.0,8.0,0.888889,10.0,1.0,1.0,...,41.0,0.694915,5.9,0.705833,6,6,9,0.666667,1.0,0.666667
2,1,10326,5.000000,28.000000,1.0,0.0,0.000000,5.0,0.0,0.0,...,41.0,0.694915,5.9,0.705833,6,6,9,0.666667,1.0,0.666667
3,1,12427,3.300000,17.600000,10.0,9.0,0.900000,10.0,1.0,1.0,...,41.0,0.694915,5.9,0.705833,6,6,9,0.666667,1.0,0.666667
4,1,13032,6.333333,21.666666,3.0,2.0,0.666667,10.0,1.0,0.0,...,41.0,0.694915,5.9,0.705833,6,6,9,0.666667,1.0,0.666667


In [25]:
df.to_pickle(root + 'datafinal.pkl')
df = pd.read_pickle(root+'datafinal.pkl')
df.head()

,user_id,product_id,avg_add_to_cart_by_user,avg_day_since_prior_order,user_total_order_product,user_total_reorder_product,user_percentage_reorder_product,last_order_in,is_reorder_3,is_reorder_2,...,user_total_reorder,user_overal_reorder,avg_order_size,mean_reorder_size,orders_3,orders_2,orders_1,reorder_3,reorder_2,reorder_1
0,1,196,1.400000,17.600000,10.0,9.0,0.900000,10.0,1.0,1.0,...,41.0,0.694915,5.9,0.705833,6,6,9,0.666667,1.0,0.666667
1,1,10258,3.333333,19.555555,9.0,8.0,0.888889,10.0,1.0,1.0,...,41.0,0.694915,5.9,0.705833,6,6,9,0.666667,1.0,0.666667
2,1,10326,5.000000,28.000000,1.0,0.0,0.000000,5.0,0.0,0.0,...,41.0,0.694915,5.9,0.705833,6,6,9,0.666667,1.0,0.666667
3,1,12427,3.300000,17.600000,10.0,9.0,0.900000,10.0,1.0,1.0,...,41.0,0.694915,5.9,0.705833,6,6,9,0.666667,1.0,0.666667
4,1,13032,6.333333,21.666666,3.0,2.0,0.666667,10.0,1.0,0.0,...,41.0,0.694915,5.9,0.705833,6,6,9,0.666667,1.0,0.666667
